In [1]:
from openai import OpenAI
from datasets import load_dataset
import json
from tqdm import tqdm

In [2]:
class DataGenerator:
    def __init__(self):
        self.openai_api_key = "EMPTY"
        self.openai_api_base = "http://192.168.1.20:1318/v1"
        self.client = OpenAI(
            api_key=self.openai_api_key,
            base_url=self.openai_api_base,
        )
        self.USE_MODEL = "pixtral"
        self.MODEL_NAME_DICT = {
            "qwen": "Qwen/Qwen2-VL-72B-Instruct",
            "pixtral": "mistralai/Pixtral-Large-Instruct-2411"
        }
        self.MODEL_NAME = self.MODEL_NAME_DICT[self.USE_MODEL]
        self.SYSTEM_PROMPT_DICT = {
            "qwen": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant.",
            "pixtral": "You are a helpful assistant."
        }
        self.SYSTEM_PROMPT = self.SYSTEM_PROMPT_DICT[self.USE_MODEL]

    def LLM_Call(self, 
                 prompt: str) -> str:
        chat_response = self.client.chat.completions.create(
            model=self.MODEL_NAME,
            messages=[
                {"role": "system", "content": self.SYSTEM_PROMPT},
                {"role": "user", "content": f"{prompt}"},
            ],
            temperature=0.001,
            top_p=0.001,
            max_tokens=4096,
            extra_body={
                "repetition_penalty": 1.03,
            },
        )
        return chat_response.choices[0].message.content

In [3]:
def get_question(context: str) -> str:
    prompt = f"""당신은 데이터 생성 전문가입니다.
    
    보안 관련 질의 데이터 1개를 만들어야합니다.
    영어로 표기해야하는 전문 용어만 영어를 사용합니다.
    영어로 표기해야하는 전문 용어가 아닐경우 무조건 질의는 한국어로 생성합니다.
    [Context]을 참고해서 질의 데이터 1개를 만들어주세요. 무조건 질의 문장만 생성합니다.
    
    [Context]
    {context}
    """
    return prompt

def get_answer(question: str,
               context: str) -> str:
    prompt = f"""당신은 데이터 생성 전문가입니다.
    
    보안 관련 응답 데이터를 만들어야합니다.
    영어로 표기해야하는 전문 용어만 영어를 사용합니다.
    영어로 표기해야하는 전문 용어가 아닐경우 무조건 응답은 한국어로 생성합니다.
    [Context]을 참고해서, [Question]에 대한 응답 데이터를 만들어주세요. 무조건 응답 문장만 생성합니다.
    
    [Context]
    {context}
    
    [Question]
    {question}
    """
    return prompt

In [4]:
data_generator = DataGenerator()

In [5]:
dataset = load_dataset("PNU-Infosec/cipher-context-dataset", "cipher", split="train")
print(dataset)

Dataset({
    features: ['output', 'system', 'instruction'],
    num_rows: 36022
})


In [6]:
parallel_dataset = dataset.select(range(0,7500))

In [13]:
JSON_DATA = []
for idx, data in enumerate(tqdm(parallel_dataset)):
    try:
        question_prompt = get_question(context=data['output'])
        question = data_generator.LLM_Call(question_prompt)
        answer_prompt = get_answer(question=question, 
                                   context=data['output'])
        answer = data_generator.LLM_Call(answer_prompt)
        JSON_DATA.append({
            "instruction": question.strip(),
            "output": answer.strip()
        })
    except:
        print(f"Error: {data}")
        continue
    if (idx % 500 == 0) or (idx == 10):
        with open(f"cipher_data_parallel_version_1.json", "w", encoding="utf-8") as f:
            json.dump(JSON_DATA, f, ensure_ascii=False, indent=4)
with open(f"cipher_data_parallel_version_1.json", "w", encoding="utf-8") as f:
    json.dump(JSON_DATA, f, ensure_ascii=False, indent=4)

  1%|          | 44/7500 [02:35<4:52:30,  2.35s/it] 

In [7]:
import json
from glob import glob
ALL_DATA = []
for path in glob("data/*.json"):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    ALL_DATA.extend(data)

with open("cipher_data_all.json", "w", encoding="utf-8") as f:
    json.dump(ALL_DATA, f, ensure_ascii=False, indent=4)